This code runs the experiment for SabreSwap v0.25 and SabreLayout v0.25


In [1]:
from qiskit.transpiler.passes.routing.sabre_swap_v020_lookahead   import SabreSwap   as SabreSwap_v020
from qiskit.transpiler.passes.layout.sabre_layout                 import SabreLayout as SabreLayout_v045
from qiskit.transpiler.passes.layout.sabre_layout_v020            import SabreLayout as SabreLayout_v020
from ulti.circuits import get_circuit_list, sort_circuits_by_depth
from ulti.coupling_maps import file_to_coupling_map
from ulti.pass_managers import generate_pass_managers, transpiled_data
import pandas as pd
import random
random.seed(42)
import os

In [2]:
# Option to keep the layout inital layout the same for faster transpilation
fast_layout = False
num_shots = 5

In [3]:
# Variables for type of circuit to test
"""
qc_names = ["ghz_2-127", "ghz_100", 
            "size5_depth_1-10", "size5_depth_10-500",
            "size25_depth_10"]
cp_names = ["hex_127", "hex_127", 
            "hex_7", "hex_7",
            "hex_27"]
"""

qc_names = ["ghz_compressed", "qvol_compressed", "size25_depth_10"]
cp_names = ["hex_127", "hex_7", "hex_27"]


for idx in range(len(qc_names)):
    qc_name = qc_names[idx]
    cp_name = cp_names[idx]

    print(f"Running experiment for the circuits: {qc_name}")
    qc_list = get_circuit_list([f'circuits/{qc_name}/'])
    qc_list = sort_circuits_by_depth(qc_list)
    
    # Experiment for Sabre v0.20 with lookahead heuristic
    look    = 15
    beam    = 10
    lp_name = "sabre_v020"
    if look < 10:
         rp_name = f"sabre_v020_look_0{look}_beam_{beam}"
    else:
        rp_name = f"sabre_v020_look_{look}_beam_{beam}"

    rp = SabreSwap_v020
    lp = SabreLayout_v020
    cm = file_to_coupling_map(f'coupling_maps/{cp_name}.txt')
    pm = generate_pass_managers(num_shots, rp, lp, cm, lookahead=look, beam_width = beam, 
                                fast_layout=fast_layout)


    csv_file_path = f'data/{qc_name}/{rp_name}.csv'
    directory = f'data/{qc_name}'
    if not os.path.exists(directory):
            os.makedirs(directory)

       # Run the experiment and store the data
    data_list = []
    for qc_idx, qc in enumerate(qc_list):
        data = transpiled_data(qc, pm)
        data['routing pass'] = rp_name
        data['layout pass'] = lp_name
        data['coupling map'] = cp_name
        data['circuit label'] = qc_idx
        data_list.append(data)

        print(f"    Finished {qc_idx} for experiment {rp_name} and circuit {qc_name}")
    print(f"Finished all for experiment {rp_name} and circuit {qc_name}")
    df = pd.DataFrame(data_list)
    df.to_csv(csv_file_path, index=False)
    print(f"Saved to {csv_file_path}")
    print("*"*50)
    print("Finished all")
    

    


Running experiment for the circuits: ghz_compressed
    Finished 0 for experiment sabre_v020_look_15_beam_10 and circuit ghz_compressed
    Finished 1 for experiment sabre_v020_look_15_beam_10 and circuit ghz_compressed
    Finished 2 for experiment sabre_v020_look_15_beam_10 and circuit ghz_compressed
    Finished 3 for experiment sabre_v020_look_15_beam_10 and circuit ghz_compressed
    Finished 4 for experiment sabre_v020_look_15_beam_10 and circuit ghz_compressed
    Finished 5 for experiment sabre_v020_look_15_beam_10 and circuit ghz_compressed
    Finished 6 for experiment sabre_v020_look_15_beam_10 and circuit ghz_compressed
    Finished 7 for experiment sabre_v020_look_15_beam_10 and circuit ghz_compressed
    Finished 8 for experiment sabre_v020_look_15_beam_10 and circuit ghz_compressed
    Finished 9 for experiment sabre_v020_look_15_beam_10 and circuit ghz_compressed
Finished all for experiment sabre_v020_look_15_beam_10 and circuit ghz_compressed
Saved to data/ghz_compress